## VISUALIZATION PROJECT Geospatial Business Intelligence (BI)


In [97]:
#Tras iniciar servidor de DB mongo, importo módulo 'pymongo'.
import pymongo
import pandas as pd
import numpy as np
import re
from pandas.io.json import json_normalize
#Asigno variable al cliente de DB y a la DB.
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [98]:
#Verifico que se han importado todos los documentos.
cursor = db.companies.find()
data = list(cursor)
print(len(data))

18801


In [99]:
def q_qty_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return len(list(map(lambda r:r,list(q))))
atr1 = 'category_code'
var1 = 'web'
print(q_qty_1(atr1,var1))

3787


In [100]:
def q_1(atr1,var1):
    q = db.companies.find({atr1:var1})
    return list(map(lambda r:[(r['name']),(r['offices'])],list(q)))
atr1 = 'category_code'
var1 = 'web'
#display(q_1(atr1,var1))

In [101]:
#Creo dataframe con los datos
df = pd.DataFrame(data)
#display(df.dtypes)
columns = df.dtypes
print(columns)

_id                     object
acquisition             object
acquisitions            object
alias_list              object
blog_feed_url           object
blog_url                object
category_code           object
competitions            object
created_at              object
crunchbase_url          object
deadpooled_day         float64
deadpooled_month       float64
deadpooled_url          object
deadpooled_year        float64
description             object
email_address           object
external_links          object
founded_day            float64
founded_month          float64
founded_year           float64
funding_rounds          object
homepage_url            object
image                   object
investments             object
ipo                     object
milestones              object
name                    object
number_of_employees    float64
offices                 object
overview                object
partners                object
permalink               object
phone_nu

In [102]:
#Extraigo columnas para tratamiento
g = df.columns.to_series().groupby(df.dtypes).groups
{k.name: v for k, v in g.items()}

{'float64': Index(['deadpooled_day', 'deadpooled_month', 'deadpooled_year', 'founded_day',
        'founded_month', 'founded_year', 'number_of_employees'],
       dtype='object'),
 'object': Index(['_id', 'acquisition', 'acquisitions', 'alias_list', 'blog_feed_url',
        'blog_url', 'category_code', 'competitions', 'created_at',
        'crunchbase_url', 'deadpooled_url', 'description', 'email_address',
        'external_links', 'funding_rounds', 'homepage_url', 'image',
        'investments', 'ipo', 'milestones', 'name', 'offices', 'overview',
        'partners', 'permalink', 'phone_number', 'products', 'providerships',
        'relationships', 'screenshots', 'tag_list', 'total_money_raised',
        'twitter_username', 'updated_at', 'video_embeds'],
       dtype='object')}

In [103]:
#Clasificación de columnas originales para limpieza del dataframe
col_float = ['deadpooled_day','deadpooled_month','deadpooled_year','founded_day','founded_month',
             'founded_year','number_of_employees']
col_obj = ['_id','acquisition','acquisitions','alias_list','blog_feed_url','blog_url','category_code',
           'competitions','created_at','crunchbase_url','deadpooled_url','description','email_address',
        'external_links','funding_rounds','homepage_url','image','investments','ipo','milestones', 
           'name','offices','overview','partners','permalink','phone_number','products','providerships',
        'relationships','screenshots','tag_list','total_money_raised','twitter_username','updated_at',
           'video_embeds']
col_drop1 = ['deadpooled_day','deadpooled_month','founded_day','founded_month','alias_list','blog_feed_url',
             'blog_url','created_at','crunchbase_url','deadpooled_url','email_address','external_links',
            'image','permalink','phone_number','screenshots','tag_list','updated_at','video_embeds']
col_drop2 = ['acquisition','acquisitions','competitions','description','funding_rounds','homepage_url',
             'investments','ipo','milestones','partners','products','providerships','relationships',
             'twitter_username','overview',]
col_ok = ['deadpooled_year','founded_year','number_of_employees','_id','category_code','name','offices',
          'total_money_raised']

print(len(col_float)+len(col_obj),len(col_drop1)+len(col_drop2)+len(col_ok))

42 42


In [104]:
#Elimino columnas con valores que no me interesan
df_drop1 = df.drop(col_drop1, axis=1)
df_drop2 = df_drop1.drop(col_drop2, axis=1)
#Elijo solo las empresas con deadpooled_year == null
df_deadnull = df_drop2[df_drop2['deadpooled_year'].isnull()]
#Elimino filas con valores nulos de las columnas que me interesan
df_nulls = df_deadnull.dropna(subset=['founded_year','number_of_employees','name','offices',
          'total_money_raised'])
display(len(df_nulls))
display(df_nulls.head())

7934

,_id,category_code,deadpooled_year,founded_year,name,number_of_employees,offices,total_money_raised
5,52cdef7c4bab8bd675297d8d,news,NaN,2004.0,Digg,60.0,"[{'description': None, 'address1': '135 Missis...",$45M
6,52cdef7c4bab8bd675297d91,web,NaN,2006.0,Geni,18.0,"[{'description': 'Headquarters', 'address1': '...",$16.5M
8,52cdef7c4bab8bd675297d93,web,NaN,1979.0,Fox Interactive Media,0.0,"[{'description': '', 'address1': '407 N Maple ...",$0
11,52cdef7c4bab8bd675297d97,news,NaN,2007.0,Scribd,50.0,"[{'description': 'HQ', 'address1': '539 Bryant...",$25.8M
12,52cdef7c4bab8bd675297d94,social,NaN,2006.0,Twitter,1300.0,"[{'description': '', 'address1': '1355 Market ...",$1.16B


In [105]:
#Elimino rows que no me aportan nada
#df_values = df_nulls[df_nulls['offices'].map(len) > 0]
#df_values = df_values[df_values.number_of_employees != 0]
#df_values = 
#print(len(df_values))
#display(df_values.head(10))

In [106]:
#Data frames por tipo de empresas y elimino las columnas _id y deadpooled_year 
col_cat = ['games_video','web','software']
df_cat = df_nulls[df_nulls['category_code'].isin(col_cat)]\
.drop(['_id','deadpooled_year'], axis=1)#.sort_values(by=['category_code','number_of_employees'])
print(len(df_cat))
display(df_cat.head())

3752


,category_code,founded_year,name,number_of_employees,offices,total_money_raised
6,web,2006.0,Geni,18.0,"[{'description': 'Headquarters', 'address1': '...",$16.5M
8,web,1979.0,Fox Interactive Media,0.0,"[{'description': '', 'address1': '407 N Maple ...",$0
16,games_video,2006.0,Joost,0.0,"[{'description': '', 'address1': '100 5th Ave ...",$45M
20,web,2002.0,Plaxo,50.0,"[{'description': 'HQ', 'address1': '1050 Enter...",$28.3M
23,web,1995.0,eBay,15000.0,"[{'description': 'Headquarters', 'address1': '...",$6.7M


In [107]:
#Elimino registros irrelevantes (tomando solo los registros que me interesan)
df_regout = df_cat.copy()
df_regout = df_regout[df_regout.number_of_employees != 0]
df_regout = df_regout[df_regout.total_money_raised != 0]
df_regout = df_regout[df_regout['offices'].map(len) > 0]
print(len(df_regout))
display(df_regout.head())

3057


,category_code,founded_year,name,number_of_employees,offices,total_money_raised
6,web,2006.0,Geni,18.0,"[{'description': 'Headquarters', 'address1': '...",$16.5M
20,web,2002.0,Plaxo,50.0,"[{'description': 'HQ', 'address1': '1050 Enter...",$28.3M
23,web,1995.0,eBay,15000.0,"[{'description': 'Headquarters', 'address1': '...",$6.7M
28,web,2007.0,Mahalo,40.0,"[{'description': '', 'address1': '3525 Eastham...",$21M
31,games_video,2006.0,Kyte,40.0,"[{'description': None, 'address1': '442 Post S...",$23.4M


In [108]:
print(df_regout['offices'])#.iloc[100])

6        [{'description': 'Headquarters', 'address1': '...
20       [{'description': 'HQ', 'address1': '1050 Enter...
23       [{'description': 'Headquarters', 'address1': '...
28       [{'description': '', 'address1': '3525 Eastham...
31       [{'description': None, 'address1': '442 Post S...
39       [{'description': None, 'address1': '632 Broadw...
48       [{'description': None, 'address1': '701 First ...
52       [{'description': 'Livestream HQ', 'address1': ...
54       [{'description': '', 'address1': '131 Varick S...
55       [{'description': 'San Francisco Office', 'addr...
56       [{'description': None, 'address1': '520 Broadw...
57       [{'description': None, 'address1': '2001 West ...
62       [{'description': '', 'address1': '1459 18th St...
64       [{'description': None, 'address1': None, 'addr...
69       [{'description': '', 'address1': '16161 Ventur...
86       [{'description': None, 'address1': '5737 Kanan...
90       [{'description': 'Arts & Crafts', 'address1': .

In [109]:
d = {'type': 'Point', 'coordinates': [None, None]}
#s = d['coordinates']
lon = d['coordinates'][0]
if lon == None:
    print('La madre que te pario')

La madre que te pario


In [110]:
#Defino funciones para la modificación de los datos de las columnas a analizar.

#Transforma un dato tipo float a uno tipo int.
def float_to_int(flt):
    integer = int(flt)
    return integer

#Transforma un dato tipo str en int devolviendo solo el número.
def str_to_int(strg):
    string = re.findall('\d+', strg )
    integer = int(string[0])*1000
    return integer

#Transforma los datos de ubicación de las oficinas en formato GeoJson y selecciona solo la primera ubicación
def nested_to_list(dicts):
    lat_lon = [{"type":"Point","coordinates":[l['longitude'],l['latitude']]} for l in dicts]
    lst = lat_lon[0]
    long = lst['coordinates'][0]
    if long != None:
        return lst
    else:
        return 0

print(nested_to_list([{'description': 'New York Office', 'address1': '', 'address2': '', 'zip_code': '', 'city': 'New York', 'state_code': None, 'country_code': 'USA', 'latitude': 40.756054, 'longitude': -73.986951},{'description': 'Barcelona Headquarters', 'address1': '', 'address2': '', 'zip_code': '', 'city': 'Barcelona', 'state_code': None, 'country_code': 'ESP', 'latitude': 41.387917, 'longitude': 2.1699187}]))

{'type': 'Point', 'coordinates': [-73.986951, 40.756054]}


In [111]:
#Transformo valores de columnas aplicando las funciones anteriormente definidas.
df_colout = df_regout.copy()
df_colout['total_money_raised'] = df_colout['total_money_raised'].apply(str_to_int)
df_colout['founded_year'] = df_colout['founded_year'].apply(float_to_int)
df_colout['offices'] = df_colout['offices'].apply(nested_to_list)
print(len(df_colout))
display(df_colout.head())

3057


,category_code,founded_year,name,number_of_employees,offices,total_money_raised
6,web,2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",16000
20,web,2002,Plaxo,50.0,"{'type': 'Point', 'coordinates': [-122.055197,...",28000
23,web,1995,eBay,15000.0,"{'type': 'Point', 'coordinates': [-121.930035,...",6000
28,web,2007,Mahalo,40.0,"{'type': 'Point', 'coordinates': [-118.487267,...",21000
31,games_video,2006,Kyte,40.0,"{'type': 'Point', 'coordinates': [-122.409173,...",23000


In [112]:
print(df_colout['offices'])

6        {'type': 'Point', 'coordinates': [-118.393064,...
20       {'type': 'Point', 'coordinates': [-122.055197,...
23       {'type': 'Point', 'coordinates': [-121.930035,...
28       {'type': 'Point', 'coordinates': [-118.487267,...
31       {'type': 'Point', 'coordinates': [-122.409173,...
39       {'type': 'Point', 'coordinates': [-73.995722, ...
48       {'type': 'Point', 'coordinates': [-122.025485,...
52       {'type': 'Point', 'coordinates': [-73.995625, ...
54       {'type': 'Point', 'coordinates': [-74.3372, 40...
55       {'type': 'Point', 'coordinates': [-122.07948, ...
56       {'type': 'Point', 'coordinates': [-73.99873, 4...
57       {'type': 'Point', 'coordinates': [-73.563878, ...
62       {'type': 'Point', 'coordinates': [-122.397224,...
64       {'type': 'Point', 'coordinates': [-119.306607,...
69       {'type': 'Point', 'coordinates': [-119.306607,...
86       {'type': 'Point', 'coordinates': [-118.756618,...
90       {'type': 'Point', 'coordinates': [4.8948623, 5.

In [113]:
#Elimino valores de GeoJson nulos
df_geoout = df_colout.copy()
df_geoout = df_geoout[df_geoout.offices != 0]
print(len(df_geoout))
display(df_geoout.head())

2305


,category_code,founded_year,name,number_of_employees,offices,total_money_raised
6,web,2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",16000
20,web,2002,Plaxo,50.0,"{'type': 'Point', 'coordinates': [-122.055197,...",28000
23,web,1995,eBay,15000.0,"{'type': 'Point', 'coordinates': [-121.930035,...",6000
28,web,2007,Mahalo,40.0,"{'type': 'Point', 'coordinates': [-118.487267,...",21000
31,games_video,2006,Kyte,40.0,"{'type': 'Point', 'coordinates': [-122.409173,...",23000


In [114]:
print(df_geoout['offices'])

6        {'type': 'Point', 'coordinates': [-118.393064,...
20       {'type': 'Point', 'coordinates': [-122.055197,...
23       {'type': 'Point', 'coordinates': [-121.930035,...
28       {'type': 'Point', 'coordinates': [-118.487267,...
31       {'type': 'Point', 'coordinates': [-122.409173,...
39       {'type': 'Point', 'coordinates': [-73.995722, ...
48       {'type': 'Point', 'coordinates': [-122.025485,...
52       {'type': 'Point', 'coordinates': [-73.995625, ...
54       {'type': 'Point', 'coordinates': [-74.3372, 40...
55       {'type': 'Point', 'coordinates': [-122.07948, ...
56       {'type': 'Point', 'coordinates': [-73.99873, 4...
57       {'type': 'Point', 'coordinates': [-73.563878, ...
62       {'type': 'Point', 'coordinates': [-122.397224,...
64       {'type': 'Point', 'coordinates': [-119.306607,...
69       {'type': 'Point', 'coordinates': [-119.306607,...
86       {'type': 'Point', 'coordinates': [-118.756618,...
90       {'type': 'Point', 'coordinates': [4.8948623, 5.

In [115]:
df_geoout.to_json('prueba.json', orient="records", lines=True)

In [146]:
def concentration(company):
    cursor = db.fase1.find({
      "offices": {
        "$near": {
          "$geometry": {
            "type": "Point",
            "coordinates": [company['offices']['coordinates'][0], company['offices']['coordinates'][1]]
          },
          "$minDistance": 0,
          "$maxDistance":9000
        }
      }
    })
    return list(cursor)

In [147]:
#Hago loop sobre las empresas
cursor_fase1 = db.fase1.find()
clusters = []
for element in cursor_fase1:
    clusters.append(len(concentration(element)))


display(len(clusters))

2305

In [148]:

print(max(clusters))


132


In [149]:
target = list(db.fase1.find())
targets = []
for i in range(len(clusters)):
    if clusters[i] == max(clusters):
        targets.append([target[i]['offices']['coordinates'][0], target[i]['offices']['coordinates'][1]])

print(targets)

[[-122.457454, 37.79851]]


In [ ]:
#df_off = json_normalize(df_cat, 'offices')
#display(df_off.head())

In [92]:
df_fase1 = pd.DataFrame(list(cursor))
print(len(df_fase1))
display(df_fase1.head())

145


,_id,category_code,founded_year,name,number_of_employees,offices,total_money_raised
0,5c6add58d5ce98ea9db48ffc,software,2003,Mantara,50.0,"{'type': 'Point', 'coordinates': [-74.032177, ...",50000
1,5c6add58d5ce98ea9db491de,web,2006,Simply Good Media,4.0,"{'type': 'Point', 'coordinates': [-74.032177, ...",0
2,5c6add58d5ce98ea9db48fff,web,2008,IAmInTown,3.0,"{'type': 'Point', 'coordinates': [-74.0348315,...",0
3,5c6add58d5ce98ea9db491b2,software,2001,Eglue Business Technologies,80.0,"{'type': 'Point', 'coordinates': [-74.02977, 4...",27000
4,5c6add58d5ce98ea9db48e49,web,2007,SuiteMatch,3.0,"{'type': 'Point', 'coordinates': [-74.039011, ...",0


In [ ]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

In [ ]:
#Esto es para buscar con regex de mongo
#{ name: { $in: [ /^key./i ] } }    { tag_list: { $in: [ /.games./i ] } }  
#{ updated_at: { $in: [ /2008$/ ] } } 
#Para descartar arrays vacios en mongo
#{offices:{ $gt: [] } } (hay otras opciones en un link en pendientes)

In [ ]:
def convertRestaurant(restaurant):
    return {
        "name": restaurant["name"],
        "cuisine":restaurant["cuisine"],
        "position": toGeoJSON(restaurant["address"]["coord"])
    }
